In [59]:
import gurobipy as grb
import pandas as pd

In [100]:
input_file="data.xlsx"
pref=pd.read_excel(input_file, sheet_name="Preferences", index_col=0)
shifts=pd.read_excel(input_file, sheet_name="Shifts", index_col=0)
week=9

In [79]:
pref.head()

,0,1,2,3,4,5,6,7,8,9,...,179,180,181,182,183,184,185,186,187,188
name,,,,,,,,,,,,,,,,,,,,,
Alexis,1,1,1,1,2,1,1,0,0,0,...,2,1,2,1,2,2,1,1,1,1
Alyssa,2,0,2,2,2,2,0,0,2,1,...,2,1,2,2,1,2,2,2,2,1
Anthony,1,1,1,2,1,2,2,2,2,2,...,2,2,1,2,1,2,1,1,2,2
Brandon,2,1,1,1,2,2,2,2,1,2,...,2,2,2,1,1,1,1,1,1,2
Brianna,2,1,2,2,1,1,1,2,2,1,...,2,1,1,1,1,1,2,1,1,2


In [81]:
shifts.head()

,day,time,persons
shift_id,,,
0,2018-04-01,Morning,15
1,2018-04-01,Evening,15
2,2018-04-01,Night,5
3,2018-04-02,Morning,15
4,2018-04-02,Evening,15


In [101]:
name=pref.index
shift=shifts.index

In [102]:
day=pd.Series(shifts.iloc[:,0], index=shift)
time=pd.Series(shifts.iloc[:, 1], index=shift)

In [103]:
mod=grb.Model()

In [104]:
x={}
for i in name:
    for j in shift:
        x[i,j]=mod.addVar(vtype=grb.GRB.BINARY, name="x[{0},{1}]".format(i, j))


In [105]:
U_s=mod.addVar(vtype=grb.GRB.INTEGER, name="max number of shifts")
L_s=mod.addVar(vtype=grb.GRB.INTEGER, name="min number of shifts")
U_n=mod.addVar(vtype=grb.GRB.INTEGER, name="max number of night shifts")
L_n=mod.addVar(vtype=grb.GRB.INTEGER, name="min number of night shifts")

In [106]:
###Set objective
mod.setObjective(sum(x[i,j]*pref.loc[i, j] for i in name for j in shift)-100*(U_s-L_s)-100*(U_n-L_s), sense=grb.GRB.MAXIMIZE)

In [107]:
####lower bound and upper bound constraints
num_shifts_u={}
num_shifts_l={}

for i in name:
    num_shifts_u[i]=mod.addConstr(U_s>=sum(x[i, j]for j in shift))
    num_shifts_l[i]=mod.addConstr(L_s<=sum(x[i, j]for j in shift))

In [108]:
num_nights_u={}
num_nights_l={}
for i in name:
    num_nights_u[i]=mod.addConstr(U_s>=sum(x[i, j]for j in shift[time=="Night"]))
    num_nights_l[i]=mod.addConstr(L_s<=sum(x[i, j]for j in shift[time=="Night"]))

In [109]:

nights_constr={}
for j in shift[time=="Night"]:
    nights_constr[j]=mod.addConstr(sum(x[i,j] for i in name)==5)

morn_eve_constr={}
for j in shift[(time=="Morning") | (time =="Evening")]:
    morn_eve_constr[j]=mod.addConstr(sum(x[i,j] for i in name)==15)


In [110]:
weeks=range(week)
week_constr={}

for i in name:
    for w in weeks:
            week_constr[i,w]=mod.addConstr(sum(x[i,j] for j in range(w*21,w*21+21))<=6)

In [116]:
###no nurse may be scheduled to consecutive shifts
consect_constrs={}

for i in name:
    for j in shift:
        if j<week*21-1:
            consect_constrs[i,j]=mod.addConstr(x[i,j]+x[i,(j+1)]<=1)
        

In [123]:
### in a night shift, 
###he/she cannot be assigned to a morning/evening shift either the day before or the day after the night shift. 

nights_bef={}
for i in name:
    for j in shift[time=="Night"]:
        if j>=2:
            nights_bef[i,j]=mod.addConstr(x[i,j]+x[i,j-2]<=1)

nights_after={}
for i in name:
    for j in shift[time=="Night"]:
        if j<week*21-1:
            nights_bef[i,j]=mod.addConstr(x[i,j]+x[i,j+2]<=1)

In [124]:
###if preference=0, x[i,j]=0
zero_constr={}
for i in name:
    for j in shift:
        if pref.loc[i, j]==0:
            zero_constr[i,j]=mod.addConstr(x[i,j]==0)

In [125]:
mod.optimize()

Optimize a model with 40373 rows, 9454 columns and 121874 nonzeros
Variable types: 0 continuous, 9454 integer (9450 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 34554 rows and 1496 columns
Presolve time: 0.22s
Presolved: 5819 rows, 7958 columns, 48041 nonzeros
Variable types: 0 continuous, 7958 integer (7956 binary)

Root relaxation: objective 1.166650e+03, 6946 iterations, 1.06 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1166.65000    0  226          - 1166.65000      -     -    4s
H    0     0                    1017.0000000 1166.65000  14.7%     -    4s
*    0     0               0    1018.0000000 1018.00000  0.00%     -    5s

Cutting planes:
  Zero half: 13

Explored 1 nodes (18778 simplex iterations) in 5.10 seconds

In [153]:
table=[]   
for i in name:
    s=[]
    for j in shift:
        if(x[i,j].x==1):
            s.append(1)
        else: s.append("")
    table.append(s)

In [154]:
df=pd.DataFrame(table, index=name, columns=shift)


In [155]:
df.to_excel("schedule.xlsx")